           CÀI ĐẶT MÔ HÌNH

In [6]:
import import_ipynb
import GRU
import emotion
import Cleanes_data
from keras.models import load_model
memotion_data = GRU.emotion_data
movie_data= Cleanes_data.movie_data




                       XÂY DỰNG CHATBOT

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Hàm để lấy danh sách phim gợi ý dựa trên cảm xúc dự đoán
def get_recommended_movies(predicted_emotion):
    # Load dữ liệu phim từ file CSV
    movie_data = pd.read_csv('D:/Nguyễn Duy Hậu/project/cleaned_movie_data.csv')
    
    # Lọc và gợi ý các bộ phim dựa trên tâm trạng dự đoán
    recommended_movies = movie_data[movie_data['emotion'] == predicted_emotion]['title'].tolist()
    
    return recommended_movies



    

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

# Loại bỏ các giá trị np.nan từ corpus
corpus = movie_data['overview'].dropna().tolist()

# Huấn luyện TfidfVectorizer từ mô tả phim
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

def prepare_input_data(user_response, tfidf_vectorizer, max_length):
    # Chuyển đổi văn bản nhập vào thành ma trận TF-IDF
    tfidf_response = tfidf_vectorizer.transform([user_response]).toarray()
    
    # Chuyển đổi ma trận TF-IDF thành định dạng phù hợp với mô hình GRU
    padded_sequence = pad_sequences(tfidf_response, maxlen=max_length, padding='post')
    
    # Reshape để phù hợp với kích thước đầu vào của mô hình GRU
    input_data = padded_sequence.reshape((1, 1, max_length))
    
    return input_data

def predict_emotion(user_response, AI_model, tfidf_vectorizer, max_length):
    # Chuẩn bị dữ liệu đầu vào
    input_data = prepare_input_data(user_response, tfidf_vectorizer, max_length)
    
    # Dự đoán cảm xúc từ dữ liệu đầu vào
    predicted_label = AI_model.predict(input_data)
    
    # Trả về nhãn cảm xúc dự đoán
    return np.argmax(predicted_label, axis=1)[0]


def chatbot():
    # Đọc mô hình GRU đã được huấn luyện từ file
    AI_model = load_model('Movie_AI.keras')
    
    # Chờ người dùng nhập câu trả lời
    user_response = input("Ngày hôm nay bạn cảm thấy thế nào? ")
    
    # Dự đoán cảm xúc từ câu trả lời của người dùng
    max_length = 60
    predicted_emotion = predict_emotion(user_response, AI_model, tfidf_vectorizer,max_length)
    print(predicted_emotion)
    
    # Lọc và gợi ý các bộ phim dựa trên cảm xúc dự đoán
    #recommended_movies = get_recommended_movies(predicted_emotion)
    
    # Hiển thị gợi ý phim cho người dùng
    #for movie in recommended_movies:
        #print(movie)

# Chạy chatbot
chatbot()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
7
